## BIOINFORMATICS THESIS: MULTIMODAL NEURAL NETWORK

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt

import os
import pickle
from tqdm.auto import tqdm
import json
from sklearn.model_selection import train_test_split

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import gensim
import random

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler
from scipy.sparse import csr_matrix
from scipy.sparse import coo_matrix, hstack
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm.auto import tqdm
import random
import types

#import optuna
import torch.nn as nn
import torch.optim as optim
import pickle
import re
import sqlite3
from sqlalchemy import create_engine



/Users/Niki/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer
import torch
import torch.nn.functional as F
import itertools
from scipy.stats import pointbiserialr
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer
from sklearn.metrics import average_precision_score
from scipy.stats import spearmanr
from scipy.stats import entropy
import optuna

class Data_Preparation():
    
    def __init__(self, data_dict, n_neighbors=5):
        
        self.label = data_dict['bed'].drop(['chrom','chromStart','chromEnd','strand'], axis=1)
        self.index = data_dict['bed'][['chrom','chromStart','chromEnd','strand']]
        self.data_dict = data_dict
        del self.data_dict['bed']
        
        self.data_dict['fa'] = self.data_dict['fa']['chromosome']
        
        # drop observations info
        for key in self.data_dict.keys():
            if key != 'fa':
                self.data_dict[key] = self.data_dict[key].drop(['chrom','chromStart','chromEnd','strand'], axis=1)
                
        
        self.n_neighbors = n_neighbors
        
        self.robust_scaler = RobustScaler()
      #  self.label_encoder = LabelEncoder()
       # self.onehot_encoder = OneHotEncoder(sparse=False) 
        self.knn_imputer = KNNImputer(n_neighbors=self.n_neighbors)
        
        self.X_train = []
        self.y_train = []
        self.X_test = []
        self.y_test = []
        
        self.sequence = []
        
        self.to_drop = dict()
    
                    
    def scale_data_genfeatures(self):
        for key in self.data_dict.keys():
            if key != 'fa':
                self.data_dict[key] = pd.DataFrame(self.robust_scaler.fit_transform(self.data_dict[key].values),
                                                   index=self.data_dict[key].index, 
                                                   columns=self.data_dict[key].columns)
                
    
    def knn_imputation_genfeatures(self):
        for key in self.data_dict.keys():
            if key != 'fa':
                self.data_dict[key] = pd.DataFrame(self.knn_imputer.fit_transform(self.data_dict[key].values),
                                                   index=self.data_dict[key].index, 
                                                   columns=self.data_dict[key].columns)
    
    
     # TRANSFORM WHEN LOADING INTO NETWORK
  #  def knn_imputation_sequence_fit(self):
   #     self.label_encoder.fit(np.array(['a','c','g','n','t']))
        
        # encode data since imputer works only with float
        #for i,seq in enumerate(self.data_dict['fa']):
         #   seq = [i for i in seq.lower()] 
          #  seq = self.label_encoder.transform(seq)
#            seq = [np.nan if i ==3 else i for i in seq]
 #           seq = self.knn_imputer.fit_transform(self.reshape(seq)).astype(int).round()
  #          self.data_dict['fa'][i] = seq
                
 #   def onehot_encode_data_sequence_fit(self):
  #      self.onehot_encoder.fit(np.array([0,1,2,4]).reshape(-1, 1)) 
        
#        for i,seq in enumerate(self.data_dict['fa']):
 #           self.data_dict['fa'][i] = self.onehot_encoder.transform(seq)
            
  #      self.data_dict['fa'] = torch.tensor(self.data_dict['fa'])
    
    def transform(self):
        
        self.scale_data_genfeatures()
        self.knn_imputation_genfeatures()
        
     #   self.knn_imputation_sequence()
      #  self.onehot_encode_data_sequence()
        
  

    def point_biserial_corr(self, X, y, verbose=False):
        """Point biserial correlation returns the correlation between a continuous and
        binary variable (target). It is a parametric test, so it assumes the data to be normally
        distributed."""
        
        uncorrelated = set()

        for col in X.columns:
            x = X[col]
            corr,_ = pointbiserialr(x,y)
            if abs(corr) < 0.05:
                uncorrelated.add(col)
                if verbose:
                    print('column: {}, Point-biserial Correlation: {}'.format(col, round(corr,4)))
        
        return uncorrelated
    
    
    def logistic_regression_corr(self, X, y, verbose=False):
        """The Logistic regression can be used to assess if a continuous variable has any 
         effect on the target. It doesn't assume anything about the distribution of the variables.
         The metric used is the area under the precision recall curve, which """
        
        uncorrelated = set()
        
        for col in X.columns:
            x = X[col].values.reshape(-1, 1)

            # perform 3-folds cv with logistic regression
            cv = KFold(n_splits=3, random_state=123, shuffle=True)
            model = LogisticRegression()

            baseline_binary_pos = len(y[y==1]) / len(y)

            # compute the AUPRC for the positive score
            AUPRC = make_scorer(average_precision_score, average='weighted')
            scores = cross_val_score(model, x, y, scoring=AUPRC, cv=cv, n_jobs=-1, error_score="raise")
        
            if scores.mean() <= baseline_binary_pos:
                uncorrelated.add(col)
                
                if verbose:
                    print('column: {}, AUPRC: {}, Baseline positive class: {}'.format(col, round(scores.mean(),4), round(baseline_binary_pos,4)))
        
        return uncorrelated

    
    def correlation_with_label(self, type_corr='all', verbose=False):
        
        if type_corr not in ['point_biserial_corr', 'logistic_regression', 'all']:
            raise ValueError(
            "Argument 'type_corr' has an incorrect value: use 'point_biserial_corr', 'logistic_regression', 'all'")
            
        
        if type_corr == 'logistic_regression':
            for key in self.data_dict.keys():
                if key != 'fa':
                    if verbose:
                        print(key)
                    self.to_drop[key] = self.logistic_regression_corr(self.data_dict[key], self.label[key], verbose=verbose)
                    # remove uncorrelated features
                    if self.to_drop[key]:
                        self.data_dict[key] = self.data_dict[key].drop(list(self.to_drop[key]), axis=1)
        
        
        elif type_corr == 'point_biserial_corr':    
            for key in self.data_dict.keys():
                if key != 'fa':
                    if verbose:
                        print(key)
                    self.to_drop[key] = self.point_biserial_corr(self.data_dict[key], self.label[key], verbose=verbose)
                    # remove uncorrelated features
                    if self.to_drop[key]:
                        self.data_dict[key] = self.data_dict[key].drop(list(self.to_drop[key]), axis=1)
        
        
        elif type_corr == 'all':
             for key in self.data_dict.keys():
                if key != 'fa':
                    if verbose:
                        print(key)
                    self.to_drop[key] = self.point_biserial_corr(self.data_dict[key], self.label[key], verbose=verbose)
                    self.to_drop[key].intersection(self.logistic_regression_corr(self.data_dict[key], self.label[key], verbose=verbose))
                    # remove uncorrelated features
                    if self.to_drop[key]:
                        self.data_dict[key] = self.data_dict[key].drop(list(self.to_drop[key]), axis=1)
        
    
    
    def spearman_corr(self, X, verbose=False):
        """Spearman correlation checks for linear correlation between continuous features.
        It is non-parametric, so normality of the variables is not necessary."""
        
        correlated = set()
        
        for col1, col2 in itertools.combinations(X.columns, 2):
            corr, _ = spearmanr(X[col1].values, X[col2].values)
            
            if corr >= 0.85:
                correlated.add(frozenset({col1,col2}))
                
                if verbose:
                    print('correlated columns: {} - {}, Spearman Correlation {}'.format(col1, col2, round(corr,4)))
        
        return correlated
    
    
    

    def remove_correlated_feature(self, X, y, correlated_pairs, verbose=False):

        correlated_to_remove = set()

        for pair in correlated_pairs:
            col1, col2 = pair
            x1 = X[col1].values.reshape(-1, 1)
            x2 = X[col2].values.reshape(-1, 1)

            # perform 3-folds cv with logistic regression
            cv = KFold(n_splits=3, random_state=123, shuffle=True)
            model = LogisticRegression()

            # compute the AUPRC for the positive score
            AUPRC = make_scorer(average_precision_score, average='weighted')
            scores1 = cross_val_score(model, x1, y, scoring=AUPRC, cv=cv, n_jobs=-1, error_score="raise")
            scores2 = cross_val_score(model, x1, y, scoring=AUPRC, cv=cv, n_jobs=-1, error_score="raise")

            if verbose:
                print('columns to compare: {} vs {}, AUPRC: {} vs {}'.format(col1, col2, scores1.mean(), scores2.mean()))

            if scores1.mean() >= scores2.mean():
                correlated_to_remove.add(col2)
            else:
                correlated_to_remove.add(col1)
    
    
    
    def correlation_btw_features(self, verbose=False):
        
        self.to_drop = dict()
        
        for key in self.data_dict.keys():
                if key != 'fa':
                    if verbose:
                        print(key)
                    correlated_pairs = self.spearman_corr(self.data_dict[key], verbose=verbose)
                    self.to_drop[key] =  self.remove_correlated_feature(self.data_dict[key], self.label[key], correlated_pairs, verbose=verbose)
                    # for each pair of correlated features, remove the one less correlated to the output
                    if self.to_drop[key]:
                        self.data_dict[key] = self.data_dict[key].drop(list(self.to_drop[key]), axis=1)
         
    
    def split_data(self, cell_line, test_size, validation_size):
        
        if self.sequence:
            assert (self.data_dict['fa'].shape[0] ==  len(self.label[cell_line]))
                    
            self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.data_dict['fa'], 
                                                                                 self.label[cell_line],
                                                                                 test_size=test_size, 
                                                                                 random_state=456, shuffle=True) 

            if self.hyper_tuning:
                assert (self.X_train.shape[0] ==  len(self.y_train))
                    
                self.X_train, self.X_test,  self.y_train, self.y_test = train_test_split(self.X_train, 
                                                                                     self.y_train,
                                                                                     test_size=validation_size,
                                                                                     random_state=123, shuffle=True) 
                   
            
        else:
            assert (self.data_dict[cell_line].shape[0] ==  len(self.label[cell_line]))

            self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.data_dict[cell_line], 
                                                                                    self.label[cell_line],
                                                                                    test_size=test_size, 
                                                                                    random_state=123, shuffle=True) 
            
            if self.hyper_tuning: 
                assert (self.X_train.shape[0] ==  len(self.y_train))

                self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X_train, 
                                                                                        self.y_train,
                                                                                        test_size=validation_size, 
                                                                                        random_state=456, shuffle=True) 
        
    
    def return_data(self, cell_line, hyper_tuning=False, sequence=True, test_size=0.25, validation_size=0.15):
        
      #  for key in self.data_dict.keys():
       #     if key != 'fa':
        #        if isinstance(self.data_dict[key], pd.DataFrame):
         #           self.data_dict[key] = torch.tensor(self.data_dict[key].values)
    
        self.sequence=sequence
        self.hyper_tuning=hyper_tuning
        
        if cell_line not in ['A549','GM12878', 'H1', 'HEK293', 'HEPG2', 'K562', 'MCF7']:
            raise ValueError(
            "Argument 'cell_line' has an incorrect value: use 'A549', 'GM12878', 'H1', 'HEK293', 'HEPG2', 'K562', 'MCF7'")
            
        
        self.split_data(cell_line=cell_line, test_size=test_size, validation_size=validation_size)
    

        return ( self.X_train.reset_index(drop=True), self.X_test.reset_index(drop=True), 
                self.y_train.reset_index(drop=True) , self.y_test.reset_index(drop=True),
                self.index )

In [4]:
from torch.utils.data import DataLoader, Dataset

class Dataset_Wrap(Dataset):
    def __init__(self, X, y, n_neighbors, sequence=False):
        super(Dataset_Wrap, self).__init__()
        
        self.X = X
        self.y = y
        self.n_neighbors = n_neighbors
        self.sequence = sequence
        
        self.pos_index = list(self.X[self.y==1].index)
        self.neg_index = list(self.X[self.y==0].index)
        
        # some observations don't have some of the nucleotides, so it will result in
        #matrices of different dimensions which cannot be concatenated
        self.label_encoder = LabelEncoder().fit(np.array(['a','c','g','n','t']))
        # get rid of value 3 which is 'n' (nan)
        self.onehot_encoder = OneHotEncoder(sparse=False).fit(np.array([0,1,2,4]).reshape(-1, 1)) 
        self.knn_imputer = KNNImputer(n_neighbors=self.n_neighbors)


    def __len__(self):
        return (self.X.shape[0])  

        
    def __getitem__(self, i):

        data = self.X.iloc[i]  
        
        if self.sequence:
            # all the letters in lowercase
            data = [i for i in data.lower()]
            
            # apply encoding
            data = self.label_encoder.transform(data)
            # value 3 corresponds to n (nan)
            data = [np.nan if i ==3 else i for i in data]
            
            # impute missing data with knn
            data =  self.knn_imputer.fit_transform( np.array(data).reshape(-1,1) ).astype(int).round()
            
            # one hot encode data
          #  data = self.onehot_encoder.transform(data)
        
        data = torch.tensor(data)
            
       # if self.sequence: 
            # (channels, size of the matrix)
        #    data = data.reshape(1, data.shape[0], data.shape[1])

        label = torch.tensor([self.y[i].astype(int)]).reshape(-1)

        return data.to(device), label.to(device)

In [5]:
from torch.utils.data import Sampler
import random

class BalancePos_BatchSampler(Sampler):
    def __init__(self, dataset, batch_size):
        
        self.pos_index = dataset.pos_index
        self.neg_index = dataset.neg_index
        
        self.batch_size = batch_size
        
        if len(dataset) % self.batch_size >0:
            self.n_batches = (len(dataset) // self.batch_size) +1
        else:
            self.n_batches = len(dataset) // self.batch_size
    
    def __iter__(self):
        random.shuffle(self.pos_index)
        random.shuffle(self.neg_index)
        
        # create chunks of positive and negative labels
        # chunks of positive labels
        pos_batches  = np.array_split(self.pos_index, self.n_batches+1)
        # chunks of negative labels
        neg_batches = np.array_split(self.neg_index, self.n_batches+1)
        neg_batches.reverse()
        
        # create batches of size 100 with the same number of positive obs. in this way we are guaranteed to have
        #evenly distributed positive observations across all the batches, so we can always apply SMOTE 
        balanced = [ np.concatenate((p_batch, n_batch)).tolist() for p_batch, n_batch in zip(pos_batches, neg_batches) ]
        random.shuffle(balanced)
        return iter(balanced)
    
    def __len__(self):
        return self.n_batches

In [30]:
import pickle

class Build_DataLoader_Pipeline():

    def __init__(self,
                 data_dict, 
                 path_name=None,
                 n_neighbors=5,
                 type_corr='all',
                 verbose=False):
    
        self.data_dict =  data_dict
        self.path_name = path_name
        self.n_neighbors = n_neighbors
        self.type_corr = type_corr
        self.verbose = verbose

        self.data_class = []
        self.transform = False
        self.correlation_with_label = False
        self.correlation_btw_features = False
        
        self.index = []
    
        
        # if there exists already the class of preprocessed data, load it
        # else instantiate it and do all the preprocessing
        if os.path.exists("data_preparation_class_{}".format(self.path_name)):
            with open("data_preparation_class_{}".format(self.path_name), "rb") as fin:
                self.data_class = pickle.load(fin)
        else:
            self.data_class = Data_Preparation(self.data_dict, n_neighbors=self.n_neighbors)
            self.data_class.transform()
            print('Data transformation Done!\n')
          #  self.data_class.correlation_with_label(type_corr=self.type_corr, verbose=self.verbose)
           # print('Check correlation with labels Done!\n')
        #    self.data_class.correlation_btw_features(verbose=self.verbose)  
         #   print('Check correlation between features Done!\n')  
            
          #  with open("data_preparation_class_{}".format(self.path_name), "wb") as fout:
           #     pickle.dump(self.data_class, fout)
        
        print('Data Preprocessing Done!')
            
        
    def return_data(self, 
                    cell_line, 
                    hyper_tuning=False, 
                    sequence=False,
                    test_size=0.25, 
                    validation_size=0.15,
                    batch_size = 100):
            
        # retrieve the data 
        X_train, X_test, y_train, y_test, index = self.data_class.return_data(cell_line=cell_line, 
                                                                          hyper_tuning=hyper_tuning, 
                                                                          sequence=sequence,
                                                                          test_size=test_size,
                                                                          validation_size=validation_size)
        
        self.index = index
        
        train_wrap = Dataset_Wrap(X_train, y_train, sequence=sequence, n_neighbors=self.n_neighbors)
        test_wrap = Dataset_Wrap(X_test, y_test, sequence=sequence, n_neighbors=self.n_neighbors)
        
        loader_train = DataLoader(dataset = train_wrap, 
                                  batch_sampler = BalancePos_BatchSampler(train_wrap, batch_size= batch_size))
        loader_test = DataLoader(dataset = test_wrap, 
                                 batch_sampler = BalancePos_BatchSampler(test_wrap, batch_size= batch_size*2)) 

        return  loader_train, loader_test

In [8]:
import pickle
with open('/content/gdrive/MyDrive/Thesis_BIOINF/enhancers_prova_pipe_data_load.pickle', "rb") as fin:
  pipe_data_load = pickle.load(fin)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RobustScaler from version 0.24.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator KNNImputer from version 0.24.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [31]:
pipe_data_load = Build_DataLoader_Pipeline(enhancers_dict, path_name='enhancers.pickle', verbose=False)

NameError: name 'enhancers_dict' is not defined

In [32]:
import pickle
with open('enhancers_prova_pipe_data_load.pickle', "rb") as fin:
  pipe_data_load = pickle.load(fin)

EOFError: Ran out of input

In [33]:
open('enhancers_prova_pipe_data_load.pickle', 'a').close()

## FUNCTIONS SETUP

In [8]:
# if the gpu is available the model is moved on the gpu memory
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [9]:
from sklearn.metrics import f1_score

def F1(output, target):
  pred = torch.argmax(output, dim=1)
  return f1_score(pred.cpu().detach().numpy(), target.cpu().detach().numpy(), average='weighted')

In [24]:
#pip install pytorchtools

In [9]:
# create a database to store optuna studies with sqlite backend

engine = create_engine('sqlite:///SA_optuna_tuning.db')

In [14]:
pip install botorch

     |████████████████████████████████| 395 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 492 kB 1.6 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
#!conda install botorch -c pytorch -c gpytorch

In [68]:
import optuna
import botorch
from optuna.integration import BoTorchSampler

class Param_Search():

  def __init__(self, 
               #network_type,
               train_loader, 
               test_loader,
               criterion,
               num_epochs,
               study_name,
               input_size,
               sequence=False, 
               n_trials=4
               ):
    # self.network_type = network_type
    self.train_loader = train_loader
    self.test_loader = test_loader
    self.criterion = criterion
    self.num_epochs = num_epochs
    self.study_name = study_name
    self.input_size = input_size
    self.sequence = sequence
    self.n_trials = n_trials
    self.best_model = None

    self.oversample_SMOTE = SMOTE(k_neighbors=3)
    self.oversample_SMOTEN = SMOTEN(k_neighbors=3)
    self.onehot_encoder = OneHotEncoder(sparse=False).fit(np.array([0,1,2,4]).reshape(-1, 1)) 
    
    """Performs the hyper parameters tuning by using a TPE (Tree-structured Parzen Estimator) 
    algorithm sampler.  
    
    Parameters:
    ------------------
    model (torch.nn.Module): neural network model.
    train_loader (DataLoader): training DataLoader object.
    test_loader (DataLoader): testing DataLoader object.
    criterion : loss function for training the model.
    num_epochs (int): number of epochs.
    study_name (str): name of the Optuna study object.
    n_trial (int): number of trials to perform in the Optuna study.
        Default: 4
    
    Attributes:
    ------------------
    best_model: stores the weights of the common layers of the best performing model.
    
    Returns:
    ------------------
    Prints values of the optimised hyperparameters and saves the parameters of the best model.
    """
    

  def objective(self, trial):
    """Defines the objective to be optimised (F1 test score) and saves
    each final model.
    """

    # generate the model
    model = FFNN_define_model(trial, in_features_INPUT=self.input_size, classes=2)

    # generate the possible optimizers
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop"])
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-1)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    # convert model data type to double
    model = model.double()

    
    # Define the training and testing phases
    for epoch in tqdm(range(1, self.num_epochs + 1), desc='Epochs'):
        train_loss = 0.0
        test_loss = 0.0
        f1_test = 0.0

        # set the model in training modality
        model.train()
        for data, target in tqdm(self.train_loader, desc='Training Model'):
            
            if self.sequence:
            #n_batches, n.elements per obs
                data = data.reshape(data.shape[0], -1)
                data,target = self.oversample_SMOTEN.fit_resample(data,target)
                # one-hot encode (must do it after oversampling)
                data_encoded = []
                for x in data:
                    data_encoded.append(self.onehot_encoder.transform(np.array(x).reshape(-1, 1)))
                # final size: n_batches, n_channels, size of matrix (256*4)
                data = torch.tensor(data_encoded).reshape(-1,1,256,4)

            else:
                data, target = self.oversample_SMOTE.fit_resample(data, target.ravel())
                data = torch.tensor(data)
        
            target = torch.tensor(target)

            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data.double())
            # calculate the batch loss as a sum of the single losses
            loss = self.criterion(output, target) 
            # backward pass: compute gradient of the loss wrt model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss
            train_loss += loss.item()
        
        # set the model in testing modality
        model.eval()
        for data, target in tqdm(self.test_loader, desc='Testing Model'):  

            if self.sequence:
                # one-hot encode (must do it after oversampling)
                data_encoded = []
                for x in data:
                    data_encoded.append(self.onehot_encoder.transform(x))
                # final size: n_batches, n_channels, size of matrix (256*4)
                data = torch.tensor(data_encoded).reshape(-1,1,256,4)
        
            target=target.reshape(-1) #########

            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data.double())
            # calculate the batch loss as a sum of the single losses
            loss = self.criterion(output, target)
            # update test loss 
            test_loss += loss.item()
            # calculate F1 test score as weighted sum of the single F1 scores
            f1_test += F1(output,target)

          # calculate epoch score by dividing by the number of observations
        f1_test /= (len(self.test_loader))
    
        # pass the score of the epoch to the study to monitor the intermediate objective values
        trial.report(f1_test, epoch)

    # save the final model named with the number of the trial 
    with open("{}{}.pickle".format(self.study_name, trial.number), "wb") as fout:
        pickle.dump(model, fout)
    
    # return F1 score to the study
    return f1_test



  def run_trial(self):
    """Runs Optuna study and stores the best model in class attribute 'best_model'."""
    
    # create a new study or load a pre-existing study. use sqlite backend to store the study.
    study = optuna.create_study(study_name=self.study_name, direction="maximize", 
                               # storage='sqlite:///SA_optuna_tuning.db', load_if_exists=True,
                                sampler=BoTorchSampler())
    
    complete_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.COMPLETE]
    pruned_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.PRUNED]
    
    # if the number of already completed trials is lower than the total number of trials passed as
    #argument, perform the remaining trials 
    if len(complete_trials)<self.n_trials:
        # set the number of trials to be performed equal to the number of missing trials
        self.n_trials -= len(complete_trials)
        study.optimize(self.objective, n_trials=self.n_trials)
        pruned_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.PRUNED]
        complete_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.COMPLETE]
        
    # store the best model found in the class
    with open("{}{}.pickle".format(self.study_name, study.best_trial.number), "rb") as fin:
        best_model = pickle.load(fin)

    self.best_model = best_model

    
    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    
    
  def save_best_model(self, path):
    """Saves the weights of the common layers of the best performing model.
    
    Parameters:
    ------------------
    path: path where the model will be stored.
    
    Returns:
    ------------------
    Weights of the common layers of the best model.
    """
    
    # retrieve the weights of the best model
    model_param = self.best_model.state_dict()
    
    # save only the weights of the common layers
    for key,value in model_param.copy().items():
        if re.findall('last', key):
            del model_param[str(key)]

    gdrive_path = '/content/gdrive/MyDrive/Thesis_BIOINF' ###
    basepath = 'models' 
    basepath = grive_path + basepath ###
    path = os.path.join(basepath, path)

    torch.save(model_param, path)

    return model_param

In [69]:
# modified from https://github.com/Bjarten/early-stopping-pytorch/blob/master/pytorchtools.py

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience.
    
    Parameters:
    ------------------
        patience (int): How long to wait after last time validation loss improved.
            Default: 7
        verbose (bool): If True, prints a message for each validation loss improvement. 
            Default: False
        delta (float): Minimum change in the monitored quantity to qualify as an improvement.
            Default: 0
        trace_func (function): trace print function.
            Default: print 
                            
    Attributes:
    ------------------
        early_stop (bool): True if the validation loss doesn't improveand the training should
            be stopped, False else.
        """
    
    def __init__(self, patience=3, verbose=False, delta=0, trace_func=print):
       
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss
        
        if self.best_score is None:
            self.best_score = score
        # if the new score is worse than the previous score, add 1 to the counter
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            # if the number of non-improving epochs is greater than patience, 
            #set to True early_stop attribute 
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0

In [16]:
import imblearn.over_sampling

In [17]:
from imblearn.over_sampling import SMOTEN

In [98]:
from tqdm.auto import tqdm
from imblearn.over_sampling import SMOTE
#from imblearn.over_sampling import SMOTEN
    

def fit(model, 
        train_loader, 
        test_loader, 
        criterion, 
        optimizer=None, 
        num_epochs=50, 
        pre_trained = False,
        filename_path=None, 
        patience=3,
        sequence=False,
        delta=0,
        verbose=True): 
    
  """Performs the training of the multitask model. It implements also early stopping
    
    Parameters:
    ------------------
    model (torch.nn.Module): neural network model.
    train_loader (DataLoader): training DataLoader object.
    test_loader (DataLoader): testing DataLoader object.
    criterion: loss function for training the model.
    optimizer (torch.optim): optimization algorithm for training the model. 
    num_epochs (int): number of epochs.
    filename_path (str): where the weights of the model at each epoch will be stored. 
        Indicate only the name of the folder.
    patience (int): number of epochs in which the test error is not anymore decreasing
        before stopping the training.
    delta (int): minimum decrease in the test error to continue with the training.
        Default:0
    verbose (bool): prints the training error, test error, F1 training score, F1 test score 
        at each epoch.
        Default: True
    
    Attributes:
    ------------------
    f1_train_scores: stores the F1 training scores for each epoch.
    f1_test_scores: stores the F1 test scores for each epoch.
    
    Returns:
    ------------------
    Lists of F1 training scores and F1 test scores at each epoch.
    Prints training error, test error, F1 training score, F1 test score at each epoch.
    """

  gdrive_path = '/content/gdrive/MyDrive/Thesis_BIOINF' ###
  basepath = 'exp'
  basepath = gdrive_path + basepath ###

  oversample_SMOTE = SMOTE(k_neighbors=3)
  #oversample_SMOTEN = imblearn.over_sampling.SMOTEN(k_neighbors=3)
  onehot_encoder = OneHotEncoder(sparse=False).fit(np.array([0,1,2,4]).reshape(-1, 1)) 


  # keep track of epoch losses 
  f1_train_scores = []
  f1_test_scores = []

  # convert model data type to double
  model = model.double()

  # define early stopping
  early_stopping = EarlyStopping(patience=patience, delta=delta, verbose=True)
    
    
  for epoch in tqdm(range(1, num_epochs + 1), desc='Epochs'):
    train_loss = 0.0
    test_loss = 0.0
    
    f1_train = 0.0
    f1_test = 0.0
    
    if pre_trained:
        pass
    
    # if there is already a trained model stored for a specific epoch, load the model
    #and don't retrain the model
    elif os.path.exists( os.path.join(basepath, filename_path + '_' + str(epoch) + '.pt') ):
      checkpoint = torch.load(PATH)
      model.load_state_dict(checkpoint['model_state_dict'])
      f1_train = checkpoint['F1_train']
      f1_test = checkpoint['F1_test']
      train_loss = checkpoint['train_loss']
      test_loss = checkpoint['test_loss']
    
    else:

      # set the model in training modality
      model.train()

      for data, target in tqdm(train_loader, desc='Training model'):
        
        # oversample using SMOTE / SMOTEN
        if sequence:
          #n_batches, n.elements per obs
          data = data.reshape(data.shape[0], -1)
          #data,target = oversample_SMOTEN.fit_resample(data,target)
          # one-hot encode (must do it after oversampling)
          data_encoded = []
          for x in data:
            data_encoded.append(onehot_encoder.transform(np.array(x).reshape(-1, 1)))
          # final size: n_batches, n_channels, size of matrix (256*4)
          data = torch.tensor(data_encoded).reshape(-1,1,256,4)

        else:
          data, target = oversample_SMOTE.fit_resample(data, target.ravel())
          data = torch.tensor(data)
        
        target = torch.tensor(target)
        
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data.double())
        # calculate the batch loss as the sum of all the losses
        loss = criterion(output, target) 
        # backward pass: compute gradient of the loss wrt model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()
        # calculate F1 training score as a weighted sum of the single F1 scores
        f1_train += F1(output,target)

        
      # set the model in testing modality
    model.eval()
    for data, target in tqdm(test_loader, desc='Testing model'):

        if sequence:
          # one-hot encode (must do it after oversampling)
          data_encoded = []
          for x in data:
            data_encoded.append(onehot_encoder.transform(x))
          # final size: n_batches, n_channels, size of matrix (256*4)
          data = torch.tensor(data_encoded).reshape(-1,1,256,4)
        
        target=target.reshape(-1)


        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data.double())
        # calculate the batch loss as the sum of all the losses
        loss = criterion(output, target)
        # update test loss
        test_loss += loss.item()
        # calculate F1 test score as a weighted sum of the single F1 scores
        f1_test += F1(output,target) 
    
    
    if pre_trained:
        continue
    else:
        # save the model weights, epoch, scores and losses at each epoch
        model_param = model.state_dict()
        PATH = os.path.join(basepath, filename_path + '_' + str(epoch) + '.pt')
        torch.save({'epoch': epoch,
                    'model_state_dict': model_param,
                    'F1_train': f1_train,
                    'F1_test': f1_test,
                    'train_loss': train_loss,
                    'test_loss': test_loss},
                   PATH)
    
    if pre_trained:
        pass
        # calculate epoch score by dividing by the number of observations
        f1_train /= (len(train_loader))
        f1_test /= (len(test_loader))
    # store epoch score
    f1_train_scores.append(f1_train)    
    f1_test_scores.append(f1_test)
      
    # print training/test statistics 
    if verbose == True:
      print('Epoch: {} \tTraining F1 score: {:.4f} \tTest F1 score: {:.4f} \tTraining Loss: {:.4f} \tTest Loss: {:.4f}'.format(
      epoch, f1_train, f1_test, train_loss, test_loss))
    
    
    if pre_trained:
        pass
    else:
        # early stop the model if the test loss is not improving
        early_stopping(test_loss, model)
        if early_stopping.early_stop:
          print('Early stopping the training')
          # reload the previous best model before the test loss started decreasing
          best_checkpoint = torch.load(os.path.join(basepath,filename_path + '_' + '{}'.format(epoch-patience) + '.pt'))
          model.load_state_dict(best_checkpoint['model_state_dict'])
          break

  
  # return the scores at each epoch
  return f1_train_scores, f1_test_scores

In [19]:
def load_model(model, path):
  """Load the stored weights of a pre-trained model into another
      model and set it to eval state.
    
    Parameters:
    ------------------
    model (torch.nn.Module): not trained neural network model.
    path (str): path of the stored weights of the pre-trained model. 
    """

  gdrive_path = '/content/gdrive/MyDrive/Thesis_BIOINF' ###
  basepath = 'models'
  basepath = gdrive_path + basepath ###

  path = os.path.join(basepath, path)
  checkpoint = torch.load(path)
  model.load_state_dict(checkpoint) 
  # set the model in testing modality
  model.eval() 

In [20]:
def save_best_model(model, path):
    """Saves only the weights of the common layers of a
    trained neural network. 
    
    Parameters:
    ------------------
    model (torch.nn.Module): trained neural network model.
    path (str): path where the weights of the trained model will be stored. 
    """
    
    model_param = model.state_dict()
    for key,value in model_param.copy().items():
      if re.findall('last', key):
        del model_param[str(key)]

    gdrive_path = '/content/gdrive/MyDrive/Thesis_BIOINF' ###
    basepath = 'models'
    basepath = gdrive_path + basepath ###
    PATH = os.path.join(basepath, path)
    
    torch.save(model_param, PATH)

In [21]:
def plot_model_scores(y_train, y_test, epochs, set_ylim=None):
    """Plots the trend of the training and test loss function of 
        a model.
    
    Parameters:
    ------------------
    y_train (list): list of training losses.
    y_test (list): list of test losses.
    epochs (int): number of epochs.
    set_ylim (tuple of int): range of y-axis.
        Default: None
    """
   
    epochs = range(epochs)
    X=pd.DataFrame({'epochs':epochs,'y_train':y_train,'y_test':y_test})
   
    sns.set_theme(style="darkgrid")
    sns.set(rc={'figure.figsize':(30,15)})

    f, ax = plt.subplots(1, 1)

    sns.lineplot(data=X, x="epochs", y="y_test", color='red',lw=2.5)
    sns.lineplot(data=X, x="epochs", y="y_train", color='green',lw=2.5)

    plt.legend(labels=['F1 test score', 'F1 train score'])
    plt.setp(ax.get_legend().get_texts(), fontsize=35)
    plt.setp(ax.get_legend().get_title(),fontsize=35)

    ax.set_ylabel('F1 score', fontsize=30)
    ax.set_xlabel('Epochs', fontsize=30)
    ax.tick_params(axis="y", labelsize=20)
    ax.tick_params(axis="x", labelsize=20)
    ax.set_ylim(set_ylim)

In [22]:
def accuracy(y_hat, y):
  pred = torch.argmax(y_hat, dim=1)
  # return the category with the highest probability
  return (pred == y).float().mean()
  # return true if the predicted category is equal to the true one, false otherwise.
  #we transform them in float, 1 if True, 0 is False, then we return the mean of the vector

# 1. FEED FORWARD NN 

In [55]:
def FFNN_define_model(trial, in_features_INPUT, classes=2):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []

    in_features = in_features_INPUT
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, in_features_INPUT)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))

        in_features = out_features

    layers.append(nn.Linear(in_features, classes))

    return nn.Sequential(*layers)

In [24]:
class FFNN(nn.Module):
  """ Feed Forward neural network. It uses ReLU activation functions."""

  def __init__(self, input_size):
    super(FFNN, self).__init__()
    self.input_size = input_size
    
    self.layer1 = nn.Sequential(
        nn.Linear(self.input_size, 100), 
        nn.ReLU())
    self.layer2 = nn.Sequential(
        nn.Linear(100, 50),
        nn.ReLU()) 
    

    self.last_layer = nn.Linear(50, 2) 
    # mat1 and mat2 shapes cannot be multiplied (190x50 and 540x100)

    self.drop_out1 = nn.Dropout(p=0.3)
    self.drop_out2 = nn.Dropout(p=0.4) 

  def forward(self, x):
      
      
    out = self.layer1(x)
    out = self.drop_out1(out)

    out = self.layer2(out)
    out = self.drop_out2(out)
    out = self.last_layer(out)

    return out

# DATA

## Hyperparameters tuning

In [26]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [49]:
train_loader, test_loader = pipe_data_load.return_data(cell_line='HEPG2', 
                    hyper_tuning=True, 
                    sequence=False)

In [35]:
def get_input_size_FFNN(data_loader):
  for d,l in data_loader:
    input_size = d.shape[1]
    break
  return input_size

In [36]:
input_size = get_input_size_FFNN(train_loader)

In [71]:
num_epochs = 2
criterion = nn.CrossEntropyLoss()

In [72]:
param_search = Param_Search(train_loader, test_loader,
            criterion, num_epochs, input_size = input_size, sequence=False,
            n_trials=2, study_name='hp_FFNN')

param_search.run_trial()

<ipython-input-68-85cc99011468>:158: ExperimentalWarning: BoTorchSampler is experimental (supported from v2.4.0). The interface can change in the future.
  sampler=BoTorchSampler())
[I 2021-06-14 14:09:42,313] A new study created in memory with name: hp_FFNN


[I 2021-06-14 14:14:36,524] Trial 0 finished with value: 0.20761762003837356 and parameters: {'n_layers': 3, 'n_units_l0': 369, 'dropout_l0': 0.4357738320752208, 'n_units_l1': 455, 'dropout_l1': 0.4073078141474769, 'n_units_l2': 347, 'dropout_l2': 0.4731143539113111, 'optimizer': 'RMSprop', 'lr': 0.04321534024331291}. Best is trial 0 with value: 0.20761762003837356.


[I 2021-06-14 14:18:30,514] Trial 1 finished with value: 0.8307348694286952 and parameters: {'n_layers': 2, 'n_units_l0': 216, 'dropout_l0': 0.3155312391750696, 'n_units_l1': 459, 'dropout_l1': 0.28529844778180163, 'optimizer': 'RMSprop', 'lr': 2.7520615182966584e-05}. Best is trial 1 with value: 0.8307348694286952.




Study statistics: 
  Number of finished trials:  2
  Number of pruned trials:  0
  Number of complete trials:  2
Best trial:
  Value:  0.8307348694286952
  Params: 
    n_layers: 2
    n_units_l0: 216
    dropout_l0: 0.3155312391750696
    n_units_l1: 459
    dropout_l1: 0.28529844778180163
    optimizer: RMSprop
    lr: 2.7520615182966584e-05


/Users/Niki/opt/anaconda3/lib/python3.8/site-packages/optuna/structs.py:18: FutureWarning: `structs` is deprecated. Classes have moved to the following modules. `structs.StudyDirection`->`study.StudyDirection`, `structs.StudySummary`->`study.StudySummary`, `structs.FrozenTrial`->`trial.FrozenTrial`, `structs.TrialState`->`trial.TrialState`, `structs.TrialPruned`->`exceptions.TrialPruned`.
  warnings.warn(_message, FutureWarning)


In [ ]:
best_model_FFNN_hp = param_search.save_best_model('FFNN/best_model_FFNN_hp.pt')

## Model testing

In [44]:
#train_loader, test_loader = pipe_data_load.return_data(cell_line='H1', 
 #                   hyper_tuning=True, 
  #                  sequence=False)

In [51]:
model=FFNN(input_size=input_size)

if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)

model.to(device)

FFNN(
  (layer1): Sequential(
    (0): Linear(in_features=540, out_features=100, bias=True)
    (1): ReLU()
  )
  (layer2): Sequential(
    (0): Linear(in_features=100, out_features=50, bias=True)
    (1): ReLU()
  )
  (last_layer): Linear(in_features=50, out_features=2, bias=True)
  (drop_out1): Dropout(p=0.3, inplace=False)
  (drop_out2): Dropout(p=0.4, inplace=False)
)

In [52]:
num_epochs = 20
criterion = nn.CrossEntropyLoss()

In [53]:
best_lr = 3.0174993222703274e-05 ##TRAIN
optimizer = optim.Adam(model.parameters(), lr=best_lr)

In [54]:
F1_train, F1_test = fit(model, train_loader, test_loader, criterion, optimizer, 
                        num_epochs, pre_trained=True, filename_path='FFNN/ffnn_testing', patience=3,
                        sequence=True, verbose=True)

#save_best_model(model, 'FFNN/best_model_FFNN_test.pt')

KeyboardInterrupt: 

In [ ]:
plot_model_scores(F1_train, F1_test, epochs=20,set_ylim=(0.82,0.88))

# 2. CONVOLUTIONAL NN

In [ ]:
def CNN_define_model(trial, in_features_INPUT, classes=2):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []

    in_channels = 1
    for i in range(n_layers):
        out_channels = trial.suggest_int("out_channels_l{}".format(i), 16, 128)
#        kernel_size = trial.suggest_int("kernel_size_l{}".format(i), 3, 15)
        layers.append( nn.Conv1d(in_channels, out_channels, kernel_size=5, stride=1, padding=1) )
        layers.append( nn.BatchNorm1d(out_channels) )
        layers.append( nn.ReLU() )
        layers.append( nn.MaxPool1d(kernel_size=3, stride=1) )

        p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))

        in_channels = out_channels

    out = out.reshape(out.size(0), -1) 
    self.last_layer1 = nn.Linear(self.fc_layer_size, 1000) 
    self.last_layer2 = nn.Linear(1000, self.classes)
    layers.append(nn.Linear(in_features, classes))

    return nn.Sequential(*layers)

In [ ]:
def __init__(self, fc_layer_size, classes=2):
    super(CNN_multitask, self).__init__()
    self.fc_layer_size = fc_layer_size 
    self.classes = classes
    
    self.layer1 = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=5, stride=1, padding=1), #The average word length in English language is 4.7 characters.
            nn.BatchNorm1d(32), 
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3, stride=1))

    self.layer2 = nn.Sequential(
            nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=1),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3, stride=1))
    
    self.layer3 = nn.Sequential(
            nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3, stride=1)
    
        
    self.drop_out1 = nn.Dropout(p=0.3)
    self.drop_out2 = nn.Dropout(p=0.4)
    self.drop_out3 = nn.Dropout(p=0.5)
    
    self.last_layer1 = nn.Linear(self.fc_layer_size, 1000) 
    self.last_layer2 = nn.Linear(1000, self.classes)
    

  def forward(self, x):
      
      # 2 shared blocks 
    out = self.layer1(x)
    out = self.drop_out1(out)
    out = self.layer2(out)
    out = self.drop_out2(out)
    out = self.layer3(out) 
    out = self.drop_out3(out)
    
    out = out.reshape(out.size(0), -1) 
    out = self.last_layer1(out)
    out = self.last_layer2(out)

 
    return out

In [73]:
pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16445 sha256=096e4125e90c900940912447fa9b888cab206e9b03a44ef24a873ffb7b0a33f4
  Stored in directory: /Users/Niki/Library/Caches/pip/wheels/84/b9/90/25a0195cf95fb5533db96f1c77ea3f296b7cc86ae8ae48e3dc
Successfully built efficientnet-pytorch
Note: you may need to restart the kernel to use updated packages.


In [82]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b7', num_classes=2)

Loaded pretrained weights for efficientnet-b7


In [83]:
model.to(device)

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 64, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(64, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        64, 64, kernel_size=(3, 3), stride=[1, 1], groups=64, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(64, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        64, 16, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        16, 64, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        64, 32, kernel_siz

In [84]:
train_loader, test_loader = pipe_data_load.return_data(cell_line='H1', 
                    hyper_tuning=True, 
                    sequence=True)

In [85]:
num_epochs = 20
criterion = nn.CrossEntropyLoss()

In [99]:
F1_train, F1_test = fit(model, train_loader, test_loader, criterion, optimizer, 
                        num_epochs, pre_trained=True, filename_path='FFNN/ffnn_testing', patience=3,
                        sequence=True, verbose=True)

#save_best_model(model, 'FFNN/best_model_FFNN_test.pt')

RuntimeError: Given groups=1, weight of size [64, 3, 3, 3], expected input[192, 1, 257, 5] to have 3 channels, but got 1 channels instead

In [97]:
F1_test

[]

# EMBRACENET APPLICATION

In [ ]:
import torch
import torch.nn as nn


class EmbraceNet(nn.Module):
  
  def __init__(self, device, input_size_list, embracement_size=256, bypass_docking=False):
    """
    Initialize an EmbraceNet module.
    Args:
      device: A "torch.device()" object to allocate internal parameters of the EmbraceNet module.
      input_size_list: A list of input sizes.
      embracement_size: The length of the output of the embracement layer ("c" in the paper).
      bypass_docking: Bypass docking step, i.e., connect the input data directly to the embracement layer. If True, input_data must have a shape of [batch_size, embracement_size].
    """
    super(EmbraceNet, self).__init__()

    self.device = device
    self.input_size_list = input_size_list
    self.embracement_size = embracement_size
    self.bypass_docking = bypass_docking

    if (not bypass_docking):
      for i, input_size in enumerate(input_size_list):
        setattr(self, 'docking_%d' % (i), nn.Linear(input_size, embracement_size))


  def forward(self, input_list, availabilities=None, selection_probabilities=None):
    """
    Forward input data to the EmbraceNet module.
    Args:
      input_list: A list of input data. Each input data should have a size as in input_size_list.
      availabilities: A 2-D tensor of shape [batch_size, num_modalities], which represents the availability of data for each modality. If None, it assumes that data of all modalities are available.
      selection_probabilities: A 2-D tensor of shape [batch_size, num_modalities], which represents probabilities that output of each docking layer will be selected ("p" in the paper). If None, the same probability of being selected will be used for each docking layer.
    Returns:
      A 2-D tensor of shape [batch_size, embracement_size] that is the embraced output.
    """

    # check input data
    assert len(input_list) == len(self.input_size_list)
    num_modalities = len(input_list)
    batch_size = input_list[0].shape[0]
    

    # docking layer
    docking_output_list = []
    if (self.bypass_docking):
      docking_output_list = input_list
    else:
      for i, input_data in enumerate(input_list):
        x = getattr(self, 'docking_%d' % (i))(input_data)
        x = nn.functional.relu(x)
        docking_output_list.append(x)
    

    # check availabilities
    if (availabilities is None):
      availabilities = torch.ones(batch_size, len(input_list), dtype=torch.float, device=self.device)
    else:
      availabilities = availabilities.float()
    

    # adjust selection probabilities
    if (selection_probabilities is None):
      selection_probabilities = torch.ones(batch_size, len(input_list), dtype=torch.float, device=self.device)
    selection_probabilities = torch.mul(selection_probabilities, availabilities)

    probability_sum = torch.sum(selection_probabilities, dim=-1, keepdim=True)
    selection_probabilities = torch.div(selection_probabilities, probability_sum)


    # stack docking outputs
    docking_output_stack = torch.stack(docking_output_list, dim=-1)  # [batch_size, embracement_size, num_modalities]


    # embrace
    modality_indices = torch.multinomial(selection_probabilities, num_samples=self.embracement_size, replacement=True)  # [batch_size, embracement_size]
    modality_toggles = nn.functional.one_hot(modality_indices, num_classes=num_modalities).float()  # [batch_size, embracement_size, num_modalities]

    embracement_output_stack = torch.mul(docking_output_stack, modality_toggles)
    embracement_output = torch.sum(embracement_output_stack, dim=-1)  # [batch_size, embracement_size]

    return embracement_output

In [ ]:
import torch.nn as nn

class VGG_pre(nn.Module):
    def __init__(self):
        super(VGG_pre, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32), 
            nn.ReLU(),
      #      nn.Conv2d(32, 32, kernel_size=5, stride=1, padding=2),
       #     nn.BatchNorm2d(32),
        #    nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
     #       nn.Conv2d(64, 64, kernel_size=5, stride=1, padding=2),
      #      nn.BatchNorm2d(64),
       #     nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.drop_out1 = nn.Dropout(p=0.3)
        self.drop_out2 = nn.Dropout(p=0.4)
       

    def forward(self, x):
      out = self.layer1(x)
      out = self.drop_out1(out)
      out = self.layer2(out)
      out = self.drop_out2(out)
      out = out.reshape(out.size(0), -1) 
      return out

In [ ]:
from collections import defaultdict

training_loader = defaultdict(lambda:0)
validation_loader = defaultdict(lambda:0)

training_loader['left'] = loader_imag_train_L
training_loader['central'] = loader_imag_train_C
training_loader['right'] = loader_imag_train_R

validation_loader['left'] = loader_imag_validation_L
validation_loader['central'] = loader_imag_validation_C
validation_loader['right'] = loader_imag_validation_R

In [ ]:
training_loader

defaultdict(<function __main__.<lambda>>,
            {'central': <torch.utils.data.dataloader.DataLoader at 0x7f0810cb6e10>,
             'left': <torch.utils.data.dataloader.DataLoader at 0x7f07ffb169d0>,
             'right': <torch.utils.data.dataloader.DataLoader at 0x7f07ffb92250>})

In [ ]:
import torch.nn as nn
import torch.optim as optim


class EmbraceNetMultimodal(nn.Module):
  def __init__(self, device, n_classes, hyperparameters_tuning=False, args=None):
    super(EmbraceNetMultimodal, self).__init__()
    
    
    # input parameters
    self.device = device
    self.n_classes = n_classes
    self.args = args
    self.hyperparameters_tuning = hyperparameters_tuning

    
    # VGG convolutional neural network
    self.VGG_left = VGG_pre()
    self.VGG_central = VGG_pre()
    self.VGG_right = VGG_pre()

    # load previously trained models to find optimal hyperparameters
    if self.hyperparameters_tuning:
      load_model(self.VGG_left, 'best_model_L_hp.pt')
      load_model(self.VGG_central, 'best_model_C_hp.pt')
      load_model(self.VGG_right, 'best_model_R_hp.pt')
    
    # load previously trained models for final testing
    else:
      load_model(self.VGG_left, 'best_model_L_test.pt')
      load_model(self.VGG_central, 'best_model_C_test.pt')
      load_model(self.VGG_right, 'best_model_R_test.pt')

    # freeze layers
    for param in self.VGG_left.parameters():
      param.requires_grad = False
    for param in self.VGG_central.parameters():
      param.requires_grad = False
    for param in self.VGG_right.parameters():
      param.requires_grad = False
        
    self.pre_output_size = (5*64*16) #5120

    # embracenet
    self.embracenet = EmbraceNet(device=self.device, 
                                 input_size_list=[self.pre_output_size, self.pre_output_size, self.pre_output_size], 
                                 embracement_size=512)

    # post embracement layers
    self.post = nn.Linear(in_features=512, out_features=self.n_classes)

  
  def forward(self, x, availabilities=None, selection_probabilities=None):

    x_left, x_central, x_right = x

    x_left_final = self.VGG_left(x_left)
    x_central_final = self.VGG_central(x_central)
    x_right_final = self.VGG_right(x_right)

    # drop left or right modality
   # availabilities = None
    #if (self.args.model_drop_left or self.args.model_drop_central or self.args.model_drop_right):
   #   availabilities = torch.ones([x.shape[0], 3], device=self.device)
    #  if (self.args.model_drop_left):
     #   availabilities[:, 0] = 0
   #   if (self.args.model_drop_central):
    #    availabilities[:, 1] = 0
     # if (self.args.model_drop_right):
      #  availabilities[:, 2] = 0

    # dropout during training
 #   if (self.is_training and self.args.model_dropout):
  #    dropout_prob = torch.rand(1, device=self.device)[0]
   #   if (dropout_prob >= 0.5):
    #    target_modalities = torch.round(torch.rand([x.shape[0]], device=self.device)).to(torch.int64)
     #  availabilities = nn.functional.one_hot(target_modalities, num_classes=2).float()

    # embrace
    embracenet = self.embracenet([x_left_final, x_central_final, x_right_final]) #, availabilities=availabilities)

    # employ final layers
    output = self.post(embracenet)

    # output softmax
    return output
    
   # return nn.functional.log_softmax(output, dim=-1) #not needed since it's already applied
   #by cross-entropy loss

In [ ]:
criterion = nn.CrossEntropyLoss()
num_epochs = 50
num_classes = 5
#learning_rate=0.0001
#optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

#optimizer = torch.optim.Adam([x_left, x_central, x_right],lr = learning_rate)

In [ ]:
import optuna
import torch.nn as nn
#import thop
import torch.optim as optim
import pickle
import re

class Param_Search_Multimodal():

  """Performs the hyper parameters tuning by using a TPE (Tree-structured Parzen Estimator) 
    algorithm sampler.  
    
    Parameters:
    ------------------
    model (torch.nn.Module): neural network model.
    train_loader (DataLoader): dictionary of training DataLoader objects. Keys of the
        dictionary must be 'FFNN', 'CNN', 'D2V_CNN'.
    test_loader (DataLoader): dictionary of testing DataLoader objects. Keys of the
        dictionary must be 'FFNN', 'CNN', 'D2V_CNN'.
    criterion : loss function for training the model.
    num_epochs (int): number of epochs.
    study_name (str): name of the Optuna study object.
    n_trial (int): number of trials to perform in the Optuna study.
        Default: 4
    
    Attributes:
    ------------------
    best_model: stores the weights of the common layers of the best performing model.
    
    Returns:
    ------------------
  Prints values of the optimised hyperparameters and saves the parameters of the best model.
    """
    
  def __init__(self, 
               model, 
               train_loader, 
               test_loader,
               criterion,
               num_epochs,
               n_trials,
               study_name):
    self.model = model
    self.train_loader = train_loader
    self.test_loader = test_loader
    self.criterion = criterion
    self.num_epochs = num_epochs
    self.n_trials = n_trials
    self.study_name = study_name
    self.len_train_loader = len(train_loader['FFNN'])
    self.len_test_loader = len(test_loader['FFNN'])
    self.best_model = None

  def objective(self, trial):
    """Defines the objective to be optimised (F1 test score) and saves
    each final model.
    """
    
    # generate the model
    model = self.model

    # generate the possible optimizers
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop"])
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-1)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    
    # convert model data type to double
    model = model.double()
    
    # Define the training and testing phases
    for epoch in tqdm(range(1, num_epochs + 1)):
      train_loss = 0.0
      test_loss = 0.0
      f1_test = 0.0
    
      # set the model in training modality
      model.train()
      for load1, load2 in tqdm(zip(self.train_loader['FFNN'],
                                          self.train_loader['CNN'])) 
                                      desc='Training model', total = self.len_train_loader):
        x_1, target = load1
        x_2, _ = load2

        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model([x_1.double(), x_2.double()])
        # calculate the batch loss as a sum of the single losses
        loss = self.criterion(output, target)
        # backward pass: compute gradient of the loss wrt model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()      
        
        
      # set the model in testing modality
      model.eval()
      for load1, load2 in tqdm(zip(self.test_loader['FFNN'],
                                          self.test_loader['CNN']), 
                                      desc='Testing model', total = self.len_test_loader):
        x_1, target = load1
        x_2, _, = load2

        # forward pass: compute predicted outputs by passing inputs to the model
        output1, output2 = model([x_1.double(), x_2.double()])
        # calculate the batch loss as a sum of the single losses
        loss = self.criterion(output, target)
        # update test loss 
        test_loss += loss.item() 
        # calculate F1 test score as weighted sum of the single F1 scores
        f1_test += F1(output,target) 
        
      # calculate epoch score by dividing by the number of observations
      f1_test /= self.len_test_loader
      # pass the score of the epoch to the study to monitor the intermediate objective values    
      trial.report(f1_test, epoch)

    # save the final model named with the number of the trial 
    with open("{}{}.pickle".format(self.study_name,trial.number), "wb") as fout:
      pickle.dump(model, fout)
    
    # return F1 score to the study        
    return f1_test



  def run_trial(self):
    """Runs Optuna study and stores the best model in class attribute 'best_model'."""

    # create a new study or load a pre-existing study. use sqlite backend to store the study.
    study = optuna.create_study(study_name=self.study_name, direction="maximize", 
                                storage='sqlite:///SA_optuna_tuning.db', load_if_exists=True)

    complete_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.COMPLETE]
    pruned_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.PRUNED]
    
    # if the number of already completed trials is lower than the total number of trials passed as
    #argument, perform the remaining trials 
    if len(complete_trials)<self.n_trials:
        # set the number of trials to be performed equal to the number of missing trials
        self.n_trials -= len(complete_trials)
        study.optimize(self.objective, n_trials=self.n_trials)
        pruned_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.PRUNED]
        complete_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.COMPLETE]
        
    # store the best model found in the class
    with open("{}{}.pickle".format(self.study_name, study.best_trial.number), "rb") as fin:
        best_model = pickle.load(fin)

    self.best_model = best_model
    
    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
      print("    {}: {}".format(key, value))
                                          

    with open("{}.pickle".format(study.best_trial.number), "rb") as fin:
      best_model = pickle.load(fin)
    
    # store only best model
    self.best_model = best_model


In [ ]:
from tqdm.auto import tqdm

def fit_multimodal(model, 
                   train_loader, 
                   test_loader, 
                   criterion, 
                   optimizer, 
                   num_epochs, 
                   filename_path, 
                   verbose=True): 
  """Performs the training of the multitask model. It implements also early stopping
    
    Parameters:
    ------------------
    model (torch.nn.Module): neural network model.
    train_loader (DataLoader): dictioary of training DataLoader objects. Keys of the
        dictionary must be 'FFNN', 'CNN', 'D2V_CNN'.
    test_loader (DataLoader): dictionary of testing DataLoader objects. Keys of the
        dictionary must be 'FFNN', 'CNN', 'D2V_CNN'.
    criterion: loss function for training the model.
    optimizer (torch.optim): optimization algorithm for training the model. 
    num_epochs (int): number of epochs.
    filename_path (str): where the weights of the model at each epoch will be stored. 
        Indicate only the name of the folder.
    patience (int): number of epochs in which the test error is not anymore decreasing
        before stopping the training.
    delta (int): minimum decrease in the test error to continue with the training.
        Default:0
    verbose (bool): prints the training error, test error, F1 training score, F1 test score 
        at each epoch.
        Default: True
    
    Attributes:
    ------------------
    f1_train_scores: stores the F1 training scores for each epoch.
    f1_test_scores: stores the F1 test scores for each epoch.
    
    Returns:
    ------------------
    Lists of F1 training scores and F1 test scores at each epoch.
    Prints training error, test error, F1 training score, F1 test score at each epoch.
    """

  basepath = 'exp'

  # keep track of epoch losses 
  f1_train_scores = []
  f1_test_scores = []

  # convert model data type to double
  model = model.double()

  # define early stopping
  early_stopping = EarlyStopping(patience=patience, delta=delta, verbose=True)

  len_train_loader = len(train_loader['FFNN'])
  len_test_loader = len(test_loader['FFNN'])
    

  for epoch in tqdm(range(1, num_epochs + 1), desc='Epochs'):
    train_loss = 0.0
    test_loss = 0.0
    
    f1_train = 0.0
    f1_test = 0.0
    
    # if there is already a trained model stored for a specific epoch, load the model
    #and don't retrain the model
    PATH = os.path.join(basepath, filename_path + '_' + str(epoch) + '.pt')
    if os.path.exists(PATH):
      checkpoint = torch.load(PATH)
      model.load_state_dict(checkpoint['model_state_dict'])
      f1_train = checkpoint['F1_train']
      f1_test = checkpoint['F1_test']
      train_loss = checkpoint['train_loss']
      test_loss = checkpoint['test_loss']
        
    else:
      # set the model in training modality
      model.train()
      for load1, load2, load3 in tqdm(zip(train_loader['FFNN'],
                                          train_loader['CNN']), 
                                      desc='Testing model', total = len_train_loader):
        x_1, target = load1
        x_2, _ = load2
        x_3, _ = load3
        
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model([x_1.double(), x_2.double()])
        # calculate the batch loss as the sum of all the losses
        loss = criterion(output1 target) 
        # backward pass: compute gradient of the loss wrt model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()
        # calculate F1 training score as a weighted sum of the single F1 scores
        f1_train +=  F1(output,target) 
        
        
      # set the model in testing modality
      model.eval()
      for load1, load2, load3 in tqdm(zip(test_loader['FFNN'],
                                          test_loader['CNN']), 
                                      desc='Testing model', total = len_test_loader):
        x_1, target = load1
        x_2, _ = load2
        x_3, _ = load3
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model([x_1.double(), x_2.double()])
        # calculate the batch loss as the sum of all the losses
        loss = criterion(output, target) 
        # update test loss
        test_loss += loss.item()
        # calculate F1 test score as a weighted sum of the single F1 scores
        f1_test +=  F1(output,target) 
        
        
    # save the model weights, epoch, scores and losses at each epoch
    model_param = model.state_dict()
    PATH = os.path.join(basepath, filename_path + '_' + str(epoch) + '.pt')
    torch.save({'epoch': epoch,
                'model_state_dict': model_param,
                'F1_train': f1_train,
                'F1_test': f1_test,
                'train_loss': train_loss,
                'test_loss': test_loss},
               PATH)
     
    
    # calculate epoch score by dividing by the number of observations
    f1_train /= len_train_loader
    f1_test /= len_test_loader
    # store epoch scores
    f1_train_scores.append(f1_train)    
    f1_test_scores.append(f1_test)
      
    # print training/test statistics 
    if verbose == True:
      print('Epoch: {} \tTraining F1 score: {:.4f} \tTest F1 score: {:.4f} \tTraining Loss: {:.4f} \tTest Loss: {:.4f}'.format(
      epoch, f1_train, f1_test, train_loss, test_loss))
      
    # early stop the model if the test loss is not improving
    early_stopping(test_loss, model)
    if early_stopping.early_stop:
      print('Early stopping the training')
      # reload the previous best model before the test loss started decreasing
      best_checkpoint = torch.load(os.path.join(basepath,filename_path + '_' + '{}'.format(epoch-patience) + '.pt'))
      model.load_state_dict(best_checkpoint['model_state_dict'])
      break
    
        
  # return the scores at each epoch
  return f1_train_scores, f1_test_scores

## Hyperparameters tuning

### Model testing